## 透過SIR模型預測美國各州新冠病毒確診人數變化
* S = 風險人口
* I = 確診人口
* R = 康復人口

![SIR](image/SIR.png)

與其去實際預測明天的ＳＩＲ，不如模擬在某某狀況下（例如：美國開始戴口罩、索國）

基本傳染數（basic reproduction number, 𝑅0）是在傳染病學上，指在沒有外力介入，同時所有人都沒有免疫力的情況下，一個感染到某種傳染病的人，會把疾病傳染給其他多少個人的平均數。基本傳染數通常被寫成為 𝑅0。𝑅0 的數字愈大，該傳染病在傳播初期的爆發潛力越強。

根據 Wikipedia: Basic Reproduction Number 數據，
SARS 的 𝑅0 約為 2 - 5，最近大爆發的武漢肺炎 COVID-19 大約在 1.4 - 3.9(平均2.65)。

![R0def](image/R0def.png)

##### 理論前提假設，沒有進行任何有效防疫措施的情況（即不考慮人口封閉、居家隔離等）
##### 各州人口(及2021/01/02統計各州總確診人口)
##### (單位：萬人)(2018/07/01估計)（僅列出確診人數前十高的人口）
1. 加利福尼亞(CA):3955(231)
2. 德克薩斯(TX):2870(177)
3. 佛羅里達(FL):2129(132)
4. 紐約(NY):1954(97.9)
5. 伊利諾伊州(IL):1274(96.6)
6. 俄亥俄州(OH):1168(70)
7. 賓夕凡尼亞(PA):1280(64.6)
8. 喬治亞(GA):1051(64.4)
9. 田納西州(TN):677(57.3)
10. 北卡羅來納(NC):1038(54.1)


In [71]:
from pylab import *
from scipy import interpolate
from random import random
from math import log
from enum import Enum

In [72]:
pop = [39550, 28700, 21290, 19540, 12740, 11680, 12800, 10510, 6770, 10380]
conf_d0 = [32, 17, 17, 15, 8, 10, 9, 11, 6, 6]
#conf_d0 = [3.22, 1.68, 1.71, 1.53, 0.79, 0.96, 0.93, 1.12, 0.61, 0.59]
#pop = np.array([3955, 2870, 2129, 1954, 1274, 1168, 1280, 1051, 677, 1038])
#conf_d0 = np.array([231, 177, 132, 98, 97, 70, 65, 64, 57, 54])
#conf_d0 = np.array([231, 177, 132, 97.9, 96.6, 70, 64.6, 64.4, 57.3, 54.1])
#pop.shape = [10, 1]
#conf_d0.shape = [10, 1]

In [73]:
for sta in range(0, 10):
    print(conf_d0[sta])
### test ###

32
17
17
15
8
10
9
11
6
6


In [ ]:
#感染事件
def InfectiousEvent(rate, dt):
    try:
        return -log(random()) / rate*1000 < dt
    except ZeroDivisionError:
        return 0

# 康復事件
def RecoveredEvent(rate, dt):
    try:
        return -log(random()) / rate*1.25 < dt
    except ZeroDivisionError:
        return 0


# Parameters
Beta = 1     ## 感染力
Gamma = 0.5  ## 康復力
#######  R0 = beta / gamma = 1 / 0.5 = 2 ###### 
day = 7     ## 模擬天數



class State(Enum):
    SUS = 1
    INF = 2
    RECV = 3


class Person:
    def __init__(self, id, state):
        self.id = id
        self.state = state

    def step(self, people, dt=1):
        if self.state == State.SUS:
            new_beta = 0

            # suppose that touch all people,
            # and add all's beta as new beta
            for person in people:
                if person.state == State.INF:
                    new_beta += Beta

            if InfectiousEvent(new_beta, dt):
                self.state = State.INF

        elif self.state == State.INF:
            if RecoveredEvent(Gamma, dt):
                self.state = State.RECV


def sir_count(people):
    s = 0
    i = 0
    r = 0
    for p in people:
        if p.state == State.SUS:
            s += 1
        elif p.state == State.INF:
            i += 1
        else:
            r += 1
    return s, i, r


for sta in range(0, 10):
    N = pop[sta]
    I0 = conf_d0[sta]
    
    if __name__ == '__main__':
        
        LS = []
        LI = []
        LR = []

        people = []
        for i in range(0, N - I0):
            people.append(Person(i, State.SUS))
        for i in range(0, I0):
            people.append(Person(i, State.INF))

        for day in range(0, day + 1):
            s, i, r = sir_count(people)
            LS.append(s)
            LI.append(i)
            LR.append(r)
            print("第{}天，易感受：{}人， 已確診：{}人， 已康復：{}人".format(day, s, i, r))
        
            for person in people:
                person.step(people)


    # Draw the figure
        t = arange(0, day + 1, 1)
        ls = interpolate.InterpolatedUnivariateSpline(t, LS)
        li = interpolate.InterpolatedUnivariateSpline(t, LI)
        lr = interpolate.InterpolatedUnivariateSpline(t, LR)
        tnew = arange(0.001, day, 1/20)
        lsnew = ls(tnew)
        linew = li(tnew)
        lrnew = lr(tnew)
        line1, = plot(tnew, lsnew, label='S')
        line2, = plot(tnew, linew, label='I')
        line3, = plot(tnew, lrnew, label='R')

        legend(handles=[line1, line2, line3],
               shadow=True, loc=(0.85, 0.4))  # handle

        line11, = plot(t, LS, '.')
        line22, = plot(t, LI, '.')
        line33, = plot(t, LR, '.')

        text(16.5, 240, 'Beta=%g' % (Beta))
        text(16.5, 190, 'Gamma=%g' % (Gamma))
        text(16.5, 150, 'Infected:%d' % (I0))
        v = [0, day, 0, N]
        axis(v)
        xlabel('time (days)')
        ylabel('Population(people)')
        title('SIR Model')
        grid(True)
        show()
